# Setup

### additional dependencies/configuration

In [ ]:
!pip install geopandas --user -q

### modules

In [1]:
import pyproj
import geopyspark as gps
import geopandas as gpd
import fiona

from functools import partial
from pyspark import SparkContext
from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, Polygon, shape
from shapely.ops import transform

In [2]:
from overlay.data import data, mask
from overlay import analysis as an

### config spark

In [3]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '8G')
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

### create overlay context

In [4]:
hou = an.Overlay('houston')

In [5]:
hou.set_study_area('/home/hadoop/notebooks/data/read/boundaries/houston.shp')

In [6]:
M.set_center(hou.study_area['centroid']['x'], hou.study_area['centroid']['y'], 9)

In [7]:
bounding = [hou.study_area['geom'][0]]

In [8]:
hou.layerset_from_poly(bounding, 1, 'base')

In [9]:
M.add_layer(hou.layersets['base'].mappable_layer)

### storm surge

In [10]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/read/overlay-layers/storm-surge/US_Category5_MOM_Inundation_HighTide.tif'
maskedtiff = '/home/hadoop/notebooks/data/write/overlay-layers/storm-surge/houston_flood_category5.tif'
mask.maskRasterFromBbox(fulltiff, hou.study_area['bbox'], maskedtiff)

In [11]:
# construct a layerset from a tiff
hou.layerset_from_tiff(maskedtiff, 'storm_surge')

In [12]:
# storm surge layer to the map
M.add_layer(hou.layersets['storm_surge'].mappable_layer)

### Experiment reclassifying NoData and adding the

In [13]:
nd = hou.layersets['storm_surge'].tiled

In [14]:
# reclassify NoData values to 1
def rc_nodata(cells, nd):
    cells[cells == nd] = 1
    return cells

addition = nd.map_cells(rc_nodata).repartition(256) + hou.layersets['base'].tiled.repartition(256)
addition_pyramid = addition.pyramid()

In [15]:
# this seems like (according to mine and James' intuition)
# it is taking too long
addition_hist = addition_pyramid.get_histogram()

In [16]:
colors = get_colors_from_matplotlib('viridis', 100)

In [17]:
add_colormap = gps.ColorMap.from_histogram(addition_hist, colors)

In [18]:
tms_layer = gps.TMS.build(addition_pyramid, add_colormap)

In [ ]:
for l in M.layers:
    M.remove_layer(l)

In [ ]:
M.add_layer(TMSRasterData(tms_layer))

You will note that this does not look exactly as we would expect. The the storm surge layer has been clipped to the extent of the study area but the NoData values from the original 'storm_surge' should be showing up as 1 (and therefore yellow) 

### sea level rise

In [ ]:
# download and unzip sea level rise polygons
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/TX/TX_HGX_slr_data_dist.zip'
d = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/"
# data.zip_to_dir(u, d)

In [ ]:
# gdb
gdb = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb"

In [ ]:
# replace with unquoted filepath to gdb in order to see layers
!ogrinfo /home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/

In [ ]:
sea_level_1 = data.read_from_gdb(gdb, 'TX_HGX_slr_1ft', MultiPolygon)

In [ ]:
hou.layerset_from_poly(sea_level_1, 1, 'sea_level')

In [ ]:
M.add_layer(hou.layersets['sea_level'].mappable_layer)

### flood

In [ ]:
# output_shp = '/home/hadoop/notebooks/data/write/overlay-layers/flood/houston_storm_surge.shp'
input_shp = '/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar.shp'

In [ ]:
clipped_shp = mask.clip_vector_to_boundaries(input_shp, hou)

In [ ]:
ct = data.read_from_shp(clipped_shp)

In [ ]:
hou.layerset_from_poly(ct, 1, 'flood')

In [ ]:
M.add_layer(hou.layersets['flood'].mappable_layer)

### testing euclidean distance (remove)

In [ ]:
euc = gps.geotrellis.euclidean_distance(sea_level_1[0], source_crs='epsg:3857', zoom=13, cell_type = gps.geotrellis.constants.CellType.FLOAT64)

In [ ]:
hou.layersets['euc'] = an.LayerSet()

In [ ]:
hou.layersets['euc'].tiled = euc

In [ ]:
hou.layersets['euc'].construct_map_layer()

In [ ]:
M.add_layer(hou.layersets['euc'].mappable_layer)

In [ ]:
len(M.layers)

### remove

In [ ]:
sl1 = data.rasterize_one_val(sea_level, 1)

In [ ]:
sl1_r = sl1.reclassify(value_map={1:1}, data_type=int, replace_nodata_with=0)

In [ ]:
sl1_reclass = an.LayerSet()

In [ ]:
sl1_reclass.build_from_poly(sea_level, 1)

In [ ]:
sl1_reclass.tiled

In [ ]:
sl1_reclass.tiled = sl1_reclass.tiled.reclassify(value_map={1:1}, data_type=int, replace_nodata_with=2)

In [ ]:
sl1_reclass.pyramid = sl1_reclass.tiled.pyramid().cache()

In [ ]:
sl1_reclass.construct_map_layer()

In [ ]:
sl1_reclass.color_map = gps.ColorMap.build({1: 0xff0000ff, 2: 0x00ff00ff})

In [ ]:
sl1_reclass.tms_layer = gps.TMS.build(sl1_reclass.pyramid, sl1_reclass.color_map)

In [ ]:
sl1_reclass.mappable_layer = TMSRasterData(sl1_reclass.tms_layer)

In [ ]:
hou.layersets['sl1_reclass'] = sl1_reclass

In [ ]:
# sl1_reclass.color_map.from_colors(color_list=)

In [ ]:
c['type'] = c.geom_type.astype(str)